In [84]:
import streamlit as st
# Langchain dependencies
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain.embeddings import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document # Importing Document schema from Langchain
from langchain.vectorstores.chroma import Chroma # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI # Import OpenAI LLM
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations

#from setting_up import chroma_collection

In [85]:
# Load environment variables from a .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [86]:
CHROMA_PATH = "chroma"
print(CHROMA_PATH)

chroma


In [87]:
#query_text = st.text_input('Enter your query:')
query_text = st.chat_input()
if not isinstance(query_text, str):
        query_text = str(query_text)

In [88]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [89]:
#I know this needs to have the other items, but I wanted to break them up.
def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
  Args:
    - query_text (str): The text to query the RAG system with.
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """

In [90]:
# YOU MUST - Use same embedding function as before
embedding_function = OpenAIEmbeddings()

# Prepare the database
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
print(db)

#This initialized the Chroma vector store from the specified directory. We need to make sure the CHROMA_PATH is set to the correct directory.

In [91]:
# # Prepare the database
# db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
# print(db)

In [92]:
# Retrieving the context from the DB using similarity search
results = db.similarity_search_with_relevance_scores(query_text, k=3)
print(results)

#This performs a similarity search using the query text and retrieces the top 'k' results

[]


In [93]:
# Check if there are any matching results or if the relevance score is too low
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

Unable to find matching results.


In [94]:
# Combine context from matching documents
context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
print(context_text)

In [95]:
# Create prompt template using context and query text
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt_template)
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question based only on the following context:\n{context}\n - -\nAnswer the question based on the above context: {question}\n'))]
Human: 
Answer the question based only on the following context:

 - -
Answer the question based on the above context: None



In [96]:
# Initialize OpenAI chat model
model = ChatOpenAI()

#This generates a response based on the context retrieved from Chroma and the provided query

In [97]:
# Generate response text based on the prompt
response_text = model.predict(prompt)
print(response_text)

I'm sorry, but it seems like there is no question provided in the context you mentioned. Could you please provide more information or clarify the question?


In [98]:
# Get sources of the matching documents
sources = [doc.metadata.get("source", None) for doc, _score in results]
print(sources)

[]


In [99]:
# Format and return response including generated text and sources
formatted_response = f"Response: {response_text}\nSources: {sources}"
#return formatted_response, response_text
print(formatted_response)

Response: I'm sorry, but it seems like there is no question provided in the context you mentioned. Could you please provide more information or clarify the question?
Sources: []


In [100]:
# # Let's call our function we have defined
formatted_response, response_text = query_rag(query_text)
# # and finally, inspect our final response!
# print(response_text)

TypeError: cannot unpack non-iterable NoneType object